In [1]:
# common imports
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
from plotly.graph_objs import *

plotly.tools.set_credentials_file(username='zhouyao0818', api_key='jK5NWZsf1kuR18tXOGla')

In [2]:
sdggg = pd.read_csv("sdggg_unit_data.csv")
sdggg.head(3)

,Series,名前,HP,EN,EXP,コスト,宇宙,空中,地上,水中,...,機動,移動,開発先A,開発先B,開発先C,開発先D,開発先A-Level,開発先B-Level,開発先C-Level,開発先D-Level
0,機動戦士ガンダム,アッガイ,8800,98,273,15900,×,×,○,○,...,17,4,ザクII,ベアッガイ,ズゴック,-,3,5,5,-
1,機動戦士ガンダム,エルメス,16500,120,471,37200,◎,×,×,×,...,19,6,ジオング,キュベレイ,-,-,4,5,-,-
2,機動戦士ガンダム,ガンキャノン,10800,102,327,20800,○,×,○,△,...,15,4,ガンダンク,ジム・キャノンII,ガンダム,-,3,4,6,-


In [3]:
sdggg.shape

(230, 22)

In [4]:
def made_graph(df, idx):
    graph = []
    melted = df.melt(id_vars=df.columns.values[idx])
    for index, row in melted.iterrows():
        if row["value"]!=' -':
            graph.append((row[df.columns.values[idx]], row["value"]))
    return graph

In [5]:
idx = [1,14,15,16,17]
sdggg_shrinked = sdggg[sdggg.columns.values[idx]]
sdggg_shrinked.head(2)

,名前,開発先A,開発先B,開発先C,開発先D
0,アッガイ,ザクII,ベアッガイ,ズゴック,-
1,エルメス,ジオング,キュベレイ,-,-


In [6]:
graph = made_graph(sdggg_shrinked,0)

In [7]:
G = nx.DiGraph()
G.add_edges_from(graph)
pos = nx.spring_layout(G)
cost = dict([(row[1],row[5])for index, row in sdggg.iterrows()])

In [8]:
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d

p=nx.single_source_shortest_path_length(G,ncenter)

In [9]:
edge_trace = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

node_trace = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[],
        size=4,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    #x, y = G.node[node]['pos']
    x, y = pos[node]
    node_trace['x'].append(x)
    node_trace['y'].append(y)

node_trace["marker"]["size"]

4

In [10]:
for node, adjacencies in enumerate(G.adjacency_list()):
    node_trace['marker']['color'].append(len(adjacencies))
#     node_info = '# of connections: '
#     node_trace['text'].append(node_info)
for node in enumerate(G.nodes()):
    node_info = node[1]
    node_trace["text"].append(node_info)

In [11]:
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title='<br>Gundam Development Graph',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
#                 arrow = True,
#                 annotations=[ dict(
#                     text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
#                     showarrow=False,
#                     xref="paper", yref="paper",
#                     x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')